In [179]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

from collections import Counter
from IPython.core.display import display, HTML
sns.set_style('darkgrid')

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import plot_confusion_matrix


 

import plotly.express as px

import warnings
warnings.filterwarnings('ignore')



import pickle
import json
import pickle
import json

In [180]:
dataset=pd.read_csv("wine_quality.csv")
dataset

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [181]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [182]:
dataset.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [183]:
dataset.rename(columns={"fixed acidity":"fixed_acidity",'volatile acidity':'volatile_acidity',
          'citric acid':'citric_acid','residual sugar':'residual_sugar',
          'free sulfur dioxide':'free_sulfur_dioxide','total sulfur dioxide':'total_sulfur_dioxide'},inplace=True)

In [191]:
dataset["quality"].value_counts()

bad     1382
good     217
Name: quality, dtype: int64

In [190]:
bins = (2, 6.5, 8)
labels = ['bad', 'good']
dataset['quality'] = pd.cut(x = dataset['quality'], bins = bins, labels = labels)

In [168]:
dataset['quality'].value_counts()

bad     1382
good     217
Name: quality, dtype: int64

In [192]:
from sklearn.preprocessing import LabelEncoder
labelencoder_y = LabelEncoder()
dataset['quality'] = labelencoder_y.fit_transform(dataset['quality'])

In [193]:
with open(r'artifacts/labelencoder_y.pkl','wb') as f:
    pickle.dump(labelencoder_y, f)

In [194]:
dataset.corr()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
fixed_acidity,1.000000,-0.256131,0.671703,0.114777,0.093705,-0.153794,-0.113181,0.668047,-0.682978,0.183006,-0.061668,0.120061
volatile_acidity,-0.256131,1.000000,-0.552496,0.001918,0.061298,-0.010504,0.076470,0.022026,0.234937,-0.260987,-0.202288,-0.270712
citric_acid,0.671703,-0.552496,1.000000,0.143577,0.203823,-0.060978,0.035533,0.364947,-0.541904,0.312770,0.109903,0.214716
residual_sugar,0.114777,0.001918,0.143577,1.000000,0.055610,0.187049,0.203028,0.355283,-0.085652,0.005527,0.042075,0.047779
chlorides,0.093705,0.061298,0.203823,0.055610,1.000000,0.005562,0.047400,0.200632,-0.265026,0.371260,-0.221141,-0.097308
free_sulfur_dioxide,-0.153794,-0.010504,-0.060978,0.187049,0.005562,1.000000,0.667666,-0.021946,0.070377,0.051658,-0.069408,-0.071747
total_sulfur_dioxide,-0.113181,0.076470,0.035533,0.203028,0.047400,0.667666,1.000000,0.071269,-0.066495,0.042947,-0.205654,-0.139517
density,0.668047,0.022026,0.364947,0.355283,0.200632,-0.021946,0.071269,1.000000,-0.341699,0.148506,-0.496180,-0.150460
pH,-0.682978,0.234937,-0.541904,-0.085652,-0.265026,0.070377,-0.066495,-0.341699,1.000000,-0.196648,0.205633,-0.057283
sulphates,0.183006,-0.260987,0.312770,0.005527,0.371260,0.051658,0.042947,0.148506,-0.196648,1.000000,0.093595,0.199485


In [195]:
dataset['quality'].value_counts()

0    1382
1     217
Name: quality, dtype: int64

In [196]:
# Feature Scaling
X=dataset.drop('quality',axis=1)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
array1 = sc.fit_transform(X)
# X_test_scaled = sc.transform(X_test)
std_df=pd.DataFrame(array1,columns=X.columns)
std_df

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
0,-0.528360,0.961877,-1.391472,-0.453218,-0.243707,-0.466193,-0.379133,0.558274,1.288643,-0.579207,-0.960246
1,-0.298547,1.967442,-1.391472,0.043416,0.223875,0.872638,0.624363,0.028261,-0.719933,0.128950,-0.584777
2,-0.298547,1.297065,-1.186070,-0.169427,0.096353,-0.083669,0.229047,0.134264,-0.331177,-0.048089,-0.584777
3,1.654856,-1.384443,1.484154,-0.453218,-0.264960,0.107592,0.411500,0.664277,-0.979104,-0.461180,-0.584777
4,-0.528360,0.961877,-1.391472,-0.453218,-0.243707,-0.466193,-0.379133,0.558274,1.288643,-0.579207,-0.960246
...,...,...,...,...,...,...,...,...,...,...,...
1594,-1.217796,0.403229,-0.980669,-0.382271,0.053845,1.542054,-0.075043,-0.978765,0.899886,-0.461180,0.072294
1595,-1.390155,0.123905,-0.877968,-0.240375,-0.541259,2.211469,0.137820,-0.862162,1.353436,0.601055,0.729364
1596,-1.160343,-0.099554,-0.723916,-0.169427,-0.243707,1.255161,-0.196679,-0.533554,0.705508,0.542042,0.541630
1597,-1.390155,0.654620,-0.775267,-0.382271,-0.264960,1.542054,-0.075043,-0.676657,1.677400,0.305990,-0.209308


In [197]:
X = dataset.drop('quality', axis = 1)
y = dataset['quality']

In [198]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [109]:
print("Shape of X_train: ",X_train.shape)
print("Shape of X_test: ", X_test.shape)
print("Shape of y_train: ",y_train.shape)
print("Shape of y_test",y_test.shape)

Shape of X_train:  (1279, 11)
Shape of X_test:  (320, 11)
Shape of y_train:  (1279, 1)
Shape of y_test (320, 1)


In [199]:
with open(r'artifacts/std_scaler.pkl','wb') as f:
    pickle.dump(sc, f)

In [200]:
x = dataset.drop('quality', axis = 1)
y = dataset.quality

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, 
                                    random_state=55, stratify=y)
log_reg_model = LogisticRegression()
log_reg_model.fit(x_train, y_train) # Gredient Descent, LogLoss, With threshold of 0.5

LogisticRegression()

In [201]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import precision_score, recall_score, f1_score

In [202]:
## Evaluation on Training Dataset

y_pred_train = log_reg_model.predict(x_train)

cnf_matrix = confusion_matrix(y_train, y_pred_train)
print("Confusion Matrix :\n", cnf_matrix)
print("*"* 50)

accuracy = accuracy_score(y_train, y_pred_train)
print("Accuracy is :",accuracy)
print("*"* 50)

clf_report = classification_report(y_train, y_pred_train)
print('Classification Report :\n',clf_report)
print("*"* 50)

precision_value = precision_score(y_train, y_pred_train)
print("Precision :",precision_value)

recall_value = recall_score(y_train, y_pred_train)
print('Recall :',recall_value)

f1_value = f1_score(y_train, y_pred_train)
print('F1 Score:',f1_value)
print("*"* 50)

Confusion Matrix :
 [[1074   31]
 [ 122   52]]
**************************************************
Accuracy is : 0.8803752931978108
**************************************************
Classification Report :
               precision    recall  f1-score   support

           0       0.90      0.97      0.93      1105
           1       0.63      0.30      0.40       174

    accuracy                           0.88      1279
   macro avg       0.76      0.64      0.67      1279
weighted avg       0.86      0.88      0.86      1279

**************************************************
Precision : 0.6265060240963856
Recall : 0.2988505747126437
F1 Score: 0.40466926070038906
**************************************************


In [203]:
clf_report = classification_report(y_train, y_pred_train)
print(clf_report)

              precision    recall  f1-score   support

           0       0.90      0.97      0.93      1105
           1       0.63      0.30      0.40       174

    accuracy                           0.88      1279
   macro avg       0.76      0.64      0.67      1279
weighted avg       0.86      0.88      0.86      1279



In [204]:
# Evaluation on Testing Dataset

y_pred = log_reg_model.predict(x_test)

cnf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix :\n", cnf_matrix)
print("*"* 50)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy is :",accuracy)
print("*"* 50)

clf_report = classification_report(y_test, y_pred)
print('Classification Report :\n',clf_report)
print("*"* 50)

precision_value = precision_score(y_test, y_pred)
print("Precision :",precision_value)

recall_value = recall_score(y_test, y_pred)
print('Recall :',recall_value)

f1_value = f1_score(y_test, y_pred)
print('F1 Score:',f1_value)
print("*"* 50)

Confusion Matrix :
 [[268   9]
 [ 31  12]]
**************************************************
Accuracy is : 0.875
**************************************************
Classification Report :
               precision    recall  f1-score   support

           0       0.90      0.97      0.93       277
           1       0.57      0.28      0.37        43

    accuracy                           0.88       320
   macro avg       0.73      0.62      0.65       320
weighted avg       0.85      0.88      0.86       320

**************************************************
Precision : 0.5714285714285714
Recall : 0.27906976744186046
F1 Score: 0.37499999999999994
**************************************************


In [205]:
clf_report = classification_report(y_test, y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.90      0.97      0.93       277
           1       0.57      0.28      0.37        43

    accuracy                           0.88       320
   macro avg       0.73      0.62      0.65       320
weighted avg       0.85      0.88      0.86       320



In [208]:
from sklearn.neighbors import KNeighborsClassifier
classifier_knn = KNeighborsClassifier()
classifier_knn.fit(x_train, y_train)


KNeighborsClassifier()

In [209]:
## Evaluation on Training Dataset

y_pred_train = classifier_knn.predict(x_train)

cnf_matrix = confusion_matrix(y_train, y_pred_train)
print("Confusion Matrix :\n", cnf_matrix)
print("*"* 50)

accuracy = accuracy_score(y_train, y_pred_train)
print("Accuracy is :",accuracy)
print("*"* 50)

clf_report = classification_report(y_train, y_pred_train)
print('Classification Report :\n',clf_report)
print("*"* 50)

precision_value = precision_score(y_train, y_pred_train)
print("Precision :",precision_value)

recall_value = recall_score(y_train, y_pred_train)
print('Recall :',recall_value)

f1_value = f1_score(y_train, y_pred_train)
print('F1 Score:',f1_value)
print("*"* 50)

Confusion Matrix :
 [[1072   33]
 [  94   80]]
**************************************************
Accuracy is : 0.9007036747458952
**************************************************
Classification Report :
               precision    recall  f1-score   support

           0       0.92      0.97      0.94      1105
           1       0.71      0.46      0.56       174

    accuracy                           0.90      1279
   macro avg       0.81      0.71      0.75      1279
weighted avg       0.89      0.90      0.89      1279

**************************************************
Precision : 0.7079646017699115
Recall : 0.45977011494252873
F1 Score: 0.5574912891986062
**************************************************


In [210]:
clf_report = classification_report(y_train, y_pred_train)
print(clf_report)

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      1105
           1       0.71      0.46      0.56       174

    accuracy                           0.90      1279
   macro avg       0.81      0.71      0.75      1279
weighted avg       0.89      0.90      0.89      1279



In [211]:
# Evaluation on Testing Dataset

y_pred = classifier_knn.predict(x_test)

cnf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix :\n", cnf_matrix)
print("*"* 50)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy is :",accuracy)
print("*"* 50)

clf_report = classification_report(y_test, y_pred)
print('Classification Report :\n',clf_report)
print("*"* 50)

precision_value = precision_score(y_test, y_pred)
print("Precision :",precision_value)

recall_value = recall_score(y_test, y_pred)
print('Recall :',recall_value)

f1_value = f1_score(y_test, y_pred)
print('F1 Score:',f1_value)
print("*"* 50)

Confusion Matrix :
 [[261  16]
 [ 32  11]]
**************************************************
Accuracy is : 0.85
**************************************************
Classification Report :
               precision    recall  f1-score   support

           0       0.89      0.94      0.92       277
           1       0.41      0.26      0.31        43

    accuracy                           0.85       320
   macro avg       0.65      0.60      0.62       320
weighted avg       0.83      0.85      0.83       320

**************************************************
Precision : 0.4074074074074074
Recall : 0.2558139534883721
F1 Score: 0.3142857142857143
**************************************************


In [212]:
clf_report = classification_report(y_train, y_pred_train)
print(clf_report)

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      1105
           1       0.71      0.46      0.56       174

    accuracy                           0.90      1279
   macro avg       0.81      0.71      0.75      1279
weighted avg       0.89      0.90      0.89      1279



In [213]:
with open(r'artifacts/regression_model.pkl','wb') as f:
    pickle.dump(log_reg_model, f)

In [155]:
log_reg_model.n_features_in_

11

In [214]:
with open(r'artifacts/knn_model.pkl','wb') as f:
    pickle.dump(classifier_knn, f)

In [215]:
classifier_knn.n_features_in_

11

In [216]:
column_names = x.columns

project_data = {"Column Names" : column_names.tolist()}

with open(r'artifacts/project_data.json','w') as f:
    json.dump(project_data, f)
    

In [217]:
log_reg_model.n_features_in_

11

In [218]:
np.zeros(11)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [219]:
np.zeros((1,11))

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])